In [ ]:
import requests  
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import mysql.connector as SQLC
import mysql.connector
from datetime import date
import time
from bs4 import BeautifulSoup

In [ ]:
#Extrair ids genebank:
data_e_hora_atuais = datetime.now()

query= input('escolha o que quer pesquisar: ')

def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
    
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    

existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
numero_genes= len(n_genes)

#Extrair ids ncbi:

Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)

#extrair links ncbi
def url_get_id(i):
    url_list= [ ]
    for id in i:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
link_genebank= url_get_id(Ids)
print(link_genebank)

#Extrair description NCBI:
description=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    description.append(info.description)

#organismos

Organismos=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Organismos.append(info.annotations['organism'])
    
#SEQ
seq=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    seq.append(f'{info.seq[0:10]}...{info.seq[-10:]}')

#percentagem de nucle:
Adenina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('A')
    perc= int(count/len(c)*100)
    Adenina.append(perc)

Citosina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('C')
    perc= int(count/len(c)*100)
    Citosina.append(perc)

Guanina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('G')
    perc= int(count/len(c)*100)
    Guanina.append(perc)

Timina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('T')
    perc= int(count/len(c)*100)
    Timina.append(perc)
    
#data
data=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    data.append(info.annotations['date'])

#len(SEQ)
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
tam=[]
for info in records:
    tam.append(len(info.seq))

# buscar info pubmed
content_id = []
for url in url_get_id(Ids):
    r_id = requests.get(url)
    content_id.append(r_id.content)
    
from bs4 import BeautifulSoup
listas=[]

for c in content_id:
    soup_id = BeautifulSoup(c, 'html.parser')

    lines = soup_id.find_all('meta', {'name':"ncbi_uidlist"} )

    id = ""
    url = ""
    for line in lines:
        if 'content' in line.attrs:
            id = line.attrs['content']
    if id:
        url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)
    r2 = requests.get(url)
    r4= r2.content.decode('utf-8')
    listas.append(r4)
    
cc= ', '.join(listas)
er= cc.replace('//','')
final= er.split(', ')

#criar dicionário de ids ncbi e ids pubmed
output_dict = {}
for x in final:
    version = re.search(r'VERSION\s+(.*?)\s', x)
    pubmed = re.search(r'PUBMED\s+(.*?)\s', x)
    if version:
        versionf=version.group(1)
        output_dict.setdefault(version.group(1), [])
    if pubmed:
        output_dict[versionf].append(pubmed.group(1))
#     else:
       # output_dict[version.group(1)].append("N/A")

#Criar Listas só com ids ncbi e ids pubmed
id_ncbii = []
ID_PUB = []
for key, vals in output_dict.items():
    if vals:
        for val in vals:
            id_ncbii.append(key)
            ID_PUB.append(val)
    else:
        id_ncbii.append(key)
        ID_PUB.append("N/A")
       


In [ ]:
#Extrair informação de artigos

titles=[]
authors=[]
source=[]
affiliation=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= ID_PUB
counter = 0
for i in idlist:
    if i!= "N/A":
        handle = Entrez.efetch(db=database, id=i, rettype="medline", retmode="text") 
        records = Medline.parse(handle)
        for info in records:
            titles.append(info.get("TI", ["N/A"]))
            authors_string = info.get("AU", ["N/A"])
            if len(authors_string) > 5:
                authors_h = authors_string[0:5]
                authors.append(authors_h)
            else:
                authors.append(authors_string) 
            source.append(info.get("SO", ["N/A"]))
            affiliation_string= info.get("AD", ["N/A"])
            if len(affiliation_string) > 5:
                affiliation_h = affiliation_string[0:5]
                affiliation.append(affiliation_h)
            else:
                affiliation.append(affiliation_string)
            counter += 1
    else:
        titles.append(["N/A"])
        authors.append(["N/A"])
        source.append(["N/A"])
        affiliation.append(["N/A"])
        counter += 1
        
#agrupar titles
titles = [ [title] for title in titles]

# agrupar dois
doi_list = []
for x in source:
    match = re.search("doi: (.*)", x)
    if match:
        doi_list.append(match.group(1))
    else:
        doi_list.append("N/A")
        
#agrupar authors
id_authors_dict = {i: authors[counter] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}        
        
pubmed_list = []
authors_list = []
for key, vals in id_authors_dict.items():
    if vals:
        for val in vals:
            pubmed_list.append(key)
            authors_list.append(val)

#agrupar affi
single_affiliation_list = []
for i in affiliation:
    single_affiliation_list.extend(i)
    

id_affiliation_dict = {i: [single_affiliation_list[counter]] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}

pubmed_affi_list = []
affi_list = []
for key, vals in id_affiliation_dict.items():
    if vals:
        for val in vals:
            pubmed_affi_list.append(key)
            affi_list.append(val)
    else:
        pubmed_affi_list.append(key)
        affi_list.append(["N/A"])

In [ ]:
#Povoação "History"

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor_2 = DataBase.cursor()
try:
    sql= "INSERT INTO History (search, Genes_number_input, Day)   VALUES (%s, %s, %s)"
    val=(query, numero_genes, data_e_hora_atuais)
    Cursor_2.execute(sql,val)
    
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
    search_id =[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor_2 = DataBase.cursor()
try:
    sql= "Select ID_search FROM History"
    Cursor_2.execute(sql)
    for row in Cursor_2:
        #print(row)
        search_id.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div= ', '.join(search_id)
h= div.replace("(",'')
hh= h.replace(",)",'')
SEARCH_ID= hh.split(', ')
Hist= SEARCH_ID[-1]

#Povoação "Gene"

import mysql.connector as SQLC
import mysql.connector

#tab = ("INSERT INTO Gene (ID_genebank) VALUES ('{}')".format(value)) # %s overwrites the same value

des = (description)
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(Ids):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO Gene (ID_genebank, Description, Organism, length, Adenina, Citosina, Guanina, Timina, Link, ID_search) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val=(value, des[index], Organismos[index], tam[index], Adenina[index], Citosina[index], Guanina[index], Timina[index], link_genebank[index], Hist)
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
#Povoar "PubMed"

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(ID_PUB):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO PubMed (ID_PumMed, title, Doi_number) VALUES (%s, %s, %s)"
        val=(str(ID_PUB[index]), str(titles[index]), str(doi_list[index]) )
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
#retirar os valores AI de pubmed
ID_AI=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_AI_PubMed FROM PubMed"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_= ', '.join(ID_AI)
h_= div_.replace("(",'')
hh_= h_.replace(",)",'')
SEARCH_ID_= hh_.split(', ')
print(SEARCH_ID_)

#Povação "Gene-Pubmed"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(id_ncbii):
        
        sql= "INSERT INTO gene_PubMed (ID_genebank, ID_AI_PubMed) VALUES (%s, %s)"
        val=(id_ncbii[index], SEARCH_ID_[index])
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
#Povação "authors"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(authors_list):
        
        sql= "INSERT INTO Authors (Name) VALUES (%s)"
        val=(authors_list[index],)
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

#retirar os valores AI de authors
ID_AI_Authors=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_Authors FROM Authors"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI_Authors.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_Authors= ', '.join(ID_AI_Authors)
h_Authors= div_Authors.replace("(",'')
hh_Authors= h_Authors.replace(",)",'')
SEARCH_ID_Authors= hh_Authors.split(', ')
print(SEARCH_ID_Authors)

#Povação "Pubmed-Authors"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(SEARCH_ID_Authors):
        
        sql= "INSERT INTO PubMed_Authors (ID_AI_PubMed, ID_Authors) VALUES (%s, %s)"
        val=(str(SEARCH_ID_[index]),str(SEARCH_ID_Authors))
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

#Povação "affi"
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(affi_list):
        
        sql= "INSERT INTO Affiliation (Info) VALUES (%s)"
        val=(affi_list[index],)
    
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

#Povoação "CDS"

import mysql.connector as SQLC
import mysql.connector


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(Ids):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO CDS (ID_CDS, Product, Translation, Location, ID_genebank, ID_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)"
        val=(get_CDS_info(Ids)[index][0], get_CDS_info(Ids)[index][2], get_CDS_info(Ids)[index][3],get_CDS_info(Ids)[index][1],join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))[index][0],join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))[index][2])
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    

#Povoação "Uniprot"
# print(uniprot_final_list) #(ID_Uniprot, Subcelular location, Function, Protein seq, length_aa)

import mysql.connector as SQLC
import mysql.connector


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
#     sql = "ALTER TABLE Uniprot AUTO_INCREMENT = 0"
#     Cursor.execute(sql)
    for index, value in enumerate(get_list_uniprot(ID_PROT)):
        #print(value)
        sql= "INSERT INTO Uniprot (ID_Uniprot, Subcelular_Location, Function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)"
        val=(get_list_uniprot(ID_PROT)[index][0],get_list_uniprot(ID_PROT)[index][1], get_list_uniprot(ID_PROT)[index][2], get_list_uniprot(ID_PROT)[index][3], get_list_uniprot(ID_PROT)[index][4], get_list_uniprot(ID_PROT)[index][5])
        Cursor.execute(sql,val)
    
    ID_AI=[]
    sql= "Select ID_AI FROM Uniprot"
    Cursor.execute(sql)
    for row in Cursor:
        ID_AI.append(str(row))
    div_= ', '.join(ID_AI)
    h_= div_.replace("(",'')
    hh_= h_.replace(",)",'')
    SEARCH_ID_AI= hh_.split(', ')
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

#para buscar os ids da tabela Uniprot, que sao AI
ID_AI=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_AI FROM Uniprot"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_= ', '.join(ID_AI)
h_= div_.replace("(",'')
hh_= h_.replace(",)",'')
SEARCH_ID_AI= hh_.split(', ')
print(SEARCH_ID_AI)